# **Classificador de Notícias com IA**
**<h3>Aluno: Antonio Vinicius Vieira Sousa</h3>**

O objetivo deste trabalho é criar um modelo de IA para classificação de noticias de acordo com a categoria

O dataset utilizado: <a href="https://www.kaggle.com/datasets/diogocaliman/notcias-publicadas-no-brasil">Historico_de_materias</a>


<h2>Instalação e importação de pacotes</h2>

  Nesse trabalho vamos utilizar as blibliotecas pandas, nltk e scikit-learn.

In [41]:
pip install pandas nltk scikit-learn

Imports e downloads necessários

In [42]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import string

# Download dos recursos do NLTK e instanciar o stemmer
nltk.download('punkt')
nltk.download('stopwords')

stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<h2>Preprocessamento dos dados</h2>
Carregar o dataset

In [43]:
# Carregar o dataset
dataframe = pd.read_csv('/content/Historico_de_materias.csv')

dataframe.head()  # Visualizar o dataset


,data,url_noticia,url_noticia_curto,titulo,conteudo_noticia,assunto
0,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/m...,"Mesmo com alta do dólar, gastos de brasileiros...","\n\tA alta de 15% no dólar em 2013, a maior do...",economia
1,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/p...,"Para Dilma, é 'apressada' a tese de que emerge...",\n\tA presidente Dilma Rousseff afirmou nesta ...,economia
2,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/t...,"Temos sido capazes de reduzir a inflação', diz...","\n\tO presidente do Banco Central, Alexandre T...",economia
3,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/g...,Governo argentino autoriza compra de dólares a...,\n\tO governo argentino anunciou nesta sexta-f...,economia
4,2014-01-25,https://web.archive.org/web/20140125123631/htt...,http://g1.globo.com/economia/noticia/2014/01/p...,Petrobras desiste de parte da área de Bem-te-v...,\n\tA Petrobras afirmou que propôs à Agência N...,economia


Definir os dados a serem usados

In [44]:
# Definir conteúdo a ser usado
dataframe['conteudo'] = dataframe['titulo'] + ' ' + dataframe['conteudo_noticia']

# Definir categorias
categorias = pd.Categorical(dataframe['assunto'])

print(categorias.categories) # Visualizar categorias
dataframe[['assunto', 'conteudo']].head() # Visualizar dataframe

Index(['economia', 'esportes', 'famosos', 'politica', 'tecnologia'], dtype='object')


,assunto,conteudo
0,economia,"Mesmo com alta do dólar, gastos de brasileiros..."
1,economia,"Para Dilma, é 'apressada' a tese de que emerge..."
2,economia,"Temos sido capazes de reduzir a inflação', diz..."
3,economia,Governo argentino autoriza compra de dólares a...
4,economia,Petrobras desiste de parte da área de Bem-te-v...


Limpeza dos dados

In [45]:
# Definir a função de pré-processamento
def preprocess(text):
    if not isinstance(text, str):  # Verificar se o valor é uma string e retornar uma string vazia se não for
      return ''

    words = word_tokenize(text.lower())  # Tokenizar o texto
    words = [ word for word in words if word.isalnum() and word not in stopwords.words('portuguese') ]  # Remover pontuação e stopwords
    words = [ stemmer.stem(word) for word in words ] # Stemmar as palavras
    return ' '.join(words)

# Aplicar a função de pré-processamento
dataframe['conteudo_processado'] = dataframe['conteudo'].apply(preprocess)

dataframe[['assunto', 'conteudo_processado']].head() # Visualizar dataframe


,assunto,conteudo_processado
0,economia,alta dólar gasto brasileiro exterior batem rec...
1,economia,dilma tese emergent perderão dinamismo preside...
2,economia,sido capaz reduzir inflação diz tombini davo p...
3,economia,governo argentino autoriza compra dólare apó q...
4,economia,petrobra desist part área amplia carcará petro...


<h2>Definir e treinar a IA</h2>

Dividir o conjunto de dados entre teino e teste

Vetorizar os dados de entrada

Definir e treinar o classificador(clf)

In [46]:
# Separar os dados em variáveis de entrada (x) e saída (y)
x = dataframe['conteudo_processado']  # Conteúdo
y = categorias.codes  # Categorias

# Dividir os dados em conjuntos de treino(80%) e teste(20%)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Ajustar e transformar os dados
vectorizer = TfidfVectorizer() # Vetorizador TF-IDF
x_train_tfidf = vectorizer.fit_transform(x_train) # Dados de treino
x_test_tfidf = vectorizer.transform(x_test) # Dados de teste

# Treinar o classificador SVM
clf = SVC(kernel='linear') # Definir o classicador
clf.fit(x_train_tfidf, y_train) # Treinar

SVC(kernel='linear')

<h2>Avaliar desempenho</h2>

Previsão do conjunto de testes

Avaliar o desempenho de acordo com os resultados

In [47]:
# Fazer previsões no conjunto de teste
y_pred = clf.predict(x_test_tfidf)

# Avaliar o desempenho do modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Acurácia: {accuracy:.4f}')
print('Relatório de Classificação:')
print(report)

Acurácia: 0.9777
Relatório de Classificação:
              precision    recall  f1-score   support

           0       0.96      0.93      0.95       318
           1       0.99      1.00      0.99      1217
           2       0.96      0.96      0.96       113
           3       0.96      0.97      0.97       259
           4       0.95      0.94      0.94       115

    accuracy                           0.98      2022
   macro avg       0.96      0.96      0.96      2022
weighted avg       0.98      0.98      0.98      2022

